# Limpeza dos Dados

In [ ]:
# Importando bibliotecas
import pandas as pd
from sqlalchemy import create_engine

server_name = 'DESKTOP-QNGLT5G'
database_name = 'Financeiro'

# Configuração da string de conexão SQLAlchemy com autenticação do Windows
conexao_sql = f'mssql+pyodbc://{server_name}/{database_name}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'

# Carregando dados do SQL Server para um DataFrame Pandas
consulta_sql = "SELECT * FROM tb_execucao_financeira"
df = pd.read_sql(consulta_sql, conexao_sql)

# 1. Tratamento de Dados Nulos
df = df.drop(['dsc_modalidade_licitacao', 'cod_item_grupo', 'dth_liquidacao'], axis=1)
df['dsc_orgao'].fillna('Desconhecido', inplace=True)
df['cod_fonte'].fillna(df['cod_fonte'].mode()[0], inplace=True)

# 2. Remoção de Colunas Desnecessárias
df = df.drop(['vlr_liquidado'], axis=1)

# 3. Conversão de Tipos de Dados
df['vlr_empenho'] = pd.to_numeric(df['vlr_empenho'], errors='coerce')

# 4. Remoção de Duplicatas
df = df.drop_duplicates()

# 5. Tratamento de Datas
df['dth_empenho'] = pd.to_datetime(df['dth_empenho'])
df['dth_pagamento'] = pd.to_datetime(df['dth_pagamento'])
df['dth_processamento'] = pd.to_datetime(df['dth_processamento'])

# Verificando novamente as informações
print(df.info())

# Salvar o DataFrame limpo para uma nova tabela no banco de dados
df.to_sql('tb_transacao_financeira', conexao_sql, index=False, if_exists='replace')


Após o uso do código acima, nós podemos ver uma diferenciação na nossa base de dados, deletando duplicatas, valores null e etc.

In [ ]:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2025116 entries, 0 to 2025115
Data columns (total 33 columns):
 #   Column               Dtype         
---  ------               -----         
 0   id                   int64         
 1   num_ano              int64         
 2   cod_ne               object        
 3   codigo_orgao         object        
 4   dsc_orgao            object        
 5   cod_credor           object        
 6   dsc_nome_credor      object        
 7   cod_fonte            object        
 8   dsc_fonte            object        
 9   cod_funcao           object        
 10  dsc_funcao           object        
 11  cod_item             object        
 12  dsc_item             object        
 13  cod_item_elemento    object        
 14  dsc_item_elemento    object        
 15  cod_item_categoria   object        
 16  dsc_item_categoria   object        
 17  dsc_item_grupo       object        
 18  cod_item_modalidade  int64         
 19  dsc_item_modalidade  object        
 20  cod_programa         object        
 21  dsc_programa         object        
 22  cod_subfuncao        object        
 23  dsc_subfuncao        object        
 24  num_sic              object        
 25  cod_np               object        
 26  vlr_empenho          float64       
 27  valor_pago           float64       
 28  vlr_resto_pagar      float64       
 29  dth_empenho          datetime64[ns]
 30  dth_pagamento        datetime64[ns]
 31  dth_processamento    datetime64[ns]
 32  num_ano_np           float64       
dtypes: datetime64[ns](3), float64(4), int64(3), object(23)
memory usage: 509.9+ MB
None